In [353]:
# Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [354]:
# !pip install opendatasets

In [355]:
# import opendatasets as od
# od.download('https://www.kaggle.com/competitions/titanic/data?select=train.csv')

In [356]:
train_data=pd.read_csv('titanic/train.csv')
test_data=pd.read_csv('titanic/test.csv')
submission_data=pd.read_csv('titanic/gender_submission.csv')

In [357]:
submission_data.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [358]:
train_data.head(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C


In [359]:
train_data.shape

(891, 12)

In [360]:
test_data.head(2)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S


In [361]:
test_data.shape

(418, 11)

In [362]:
y=train_data[['Survived']]
X=train_data.drop(['Survived'],axis=1)
X=pd.concat([X,test_data])

In [363]:
X.head(5)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [364]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  1309 non-null   int64  
 1   Pclass       1309 non-null   int64  
 2   Name         1309 non-null   object 
 3   Sex          1309 non-null   object 
 4   Age          1046 non-null   float64
 5   SibSp        1309 non-null   int64  
 6   Parch        1309 non-null   int64  
 7   Ticket       1309 non-null   object 
 8   Fare         1308 non-null   float64
 9   Cabin        295 non-null    object 
 10  Embarked     1307 non-null   object 
dtypes: float64(2), int64(4), object(5)
memory usage: 122.7+ KB


In [365]:
X=X.drop(['Cabin','Name','Ticket'],axis=1)
X.columns

Index(['PassengerId', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare',
       'Embarked'],
      dtype='object')

# Explore categorical variables

In [366]:
categorical = [var for var in X if X[var].dtype=='O']
print('There are {} categorical variable'.format(len(categorical)))
print('Categorical Variable are:\n\n',categorical)

There are 2 categorical variable
Categorical Variable are:

 ['Sex', 'Embarked']


In [367]:
X[categorical].head(3)

,Sex,Embarked
0,male,S
1,female,C
2,female,S


In [368]:
X[categorical].isnull().sum()

Sex         0
Embarked    2
dtype: int64

In [369]:
for var in categorical:
  print(X[var].value_counts())
  print('*'*30)

male      843
female    466
Name: Sex, dtype: int64
******************************
S    914
C    270
Q    123
Name: Embarked, dtype: int64
******************************


In [370]:
for var in categorical:
  print(var, ' contains ', len(X[var].unique()), ' labels')

Sex  contains  2  labels
Embarked  contains  4  labels


In [371]:
X['Embarked'].unique()

array(['S', 'C', 'Q', nan], dtype=object)

In [372]:
X['Embarked'].value_counts(normalize=True)*100

S    69.931140
C    20.657995
Q     9.410865
Name: Embarked, dtype: float64

In [373]:
X['Embarked'].fillna(X['Embarked'].mode()[0],inplace=True)

In [374]:
X[categorical].head()

,Sex,Embarked
0,male,S
1,female,C
2,female,S
3,female,S
4,male,S


# Explore Numerical Variables

In [375]:
numerical=[var for var in X if X[var].dtype!='O']

In [376]:
X[numerical].head()

,PassengerId,Pclass,Age,SibSp,Parch,Fare
0,1,3,22.0,1,0,7.2500
1,2,1,38.0,1,0,71.2833
2,3,3,26.0,0,0,7.9250
3,4,1,35.0,1,0,53.1000
4,5,3,35.0,0,0,8.0500


In [377]:
X['Age'].fillna(X['Age'].mean(),inplace=True)
X['Fare'].fillna(X['Fare'].mean(),inplace=True)

In [378]:
X[numerical].isnull().sum()

PassengerId    0
Pclass         0
Age            0
SibSp          0
Parch          0
Fare           0
dtype: int64

In [379]:
age_labels = ['0-16', '16-32', '32-48', '48-64','64-80']
X['Age_encoding'] = pd.cut(X['Age'], bins=5, labels=age_labels, right=False)
X=X.drop(['Age'],axis=1)

In [380]:
numerical=[var for var in X if X[var].dtype!='O']
X[numerical].head(2)

,PassengerId,Pclass,SibSp,Parch,Fare,Age_encoding
0,1,3,1,0,7.2500,16-32
1,2,1,1,0,71.2833,32-48


In [381]:
fare_labels=['0-102','102-204','204-307','307-409','409-512']
X['Fare_interval']=pd.cut(X['Fare'],bins=5,labels=fare_labels,right=False)
X=X.drop(['Fare'],axis=1)
numerical=[var for var in X if X[var].dtype!='O']
X[numerical].head(2)

,PassengerId,Pclass,SibSp,Parch,Age_encoding,Fare_interval
0,1,3,1,0,16-32,0-102
1,2,1,1,0,32-48,0-102


In [382]:
X['Sex']=X['Sex'].replace({'male':0,'female':1})
X['Embarked']=X['Embarked'].replace({'S':0,'C':1,'Q':2})

In [383]:
X['Age_encoding']=X['Age_encoding'].astype('O')
X['Fare_interval']=X['Fare_interval'].astype('O')

In [384]:
X['Embarked']=X['Embarked'].replace({'S':0,'C':1,'Q':2})
X['Age_encoding']=X['Age_encoding'].replace({'0-16':0,'16-32':1,'32-48':2,'48-64':3,'64-80':4})
X['Fare_interval']=X['Fare_interval'].replace({'0-102':0,'102-204':1,'204-307':2,'409-512':3})

In [385]:
X.isnull().sum()

PassengerId      0
Pclass           0
Sex              0
SibSp            0
Parch            0
Embarked         0
Age_encoding     0
Fare_interval    0
dtype: int64

In [386]:
X.shape

(1309, 8)

In [387]:
y.shape

(891, 1)

In [388]:
train_data.shape

(891, 12)

In [389]:
test_data.shape

(418, 11)

In [390]:
train_df=X[0:891]
test_df=X[891:]

In [391]:
X_train=train_df.drop('PassengerId',axis=1)
y_train=y['Survived']
X_test=test_df.drop('PassengerId',axis=1)
X_train.shape,y_train.shape,X_test.shape

((891, 7), (891,), (418, 7))

In [414]:
# Logistic Regression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

params=[{'C':[1,5,10]},{'max_iter':[100,150]}]
clf=LogisticRegression(C=100,max_iter=100).fit(X_train,y_train)
y_pred=clf.predict(X_test)
print(y_pred)
print('-'*100)
model=GridSearchCV(clf,param_grid=params,scoring='f1',cv=5)
model.fit(X_train,y_train)
print(model.best_params_)
print(model.best_score_)

[0 0 0 0 1 0 1 0 1 0 0 0 1 0 1 1 0 0 1 1 0 0 1 1 1 0 1 0 0 0 0 0 0 1 1 0 1
 1 0 0 0 1 0 1 1 0 0 0 1 0 0 0 1 1 0 0 0 0 0 1 0 0 0 1 1 1 1 0 1 1 1 0 1 1
 1 0 0 1 0 1 0 0 0 0 0 0 1 1 1 0 1 0 1 0 1 0 1 0 1 0 1 0 0 0 1 0 0 0 0 0 0
 1 1 1 1 0 0 1 0 1 1 0 1 0 0 1 0 1 0 0 0 0 1 0 0 0 0 0 1 0 0 1 0 0 0 0 1 0
 1 0 1 0 0 0 0 0 1 1 0 1 1 0 1 0 0 1 0 0 1 1 0 0 0 0 0 1 1 0 1 1 0 0 1 0 1
 0 1 0 0 0 0 1 0 0 0 0 1 1 0 1 1 0 0 1 0 1 1 0 1 0 0 0 0 1 0 0 1 0 1 0 1 0
 1 0 1 1 0 1 0 0 0 1 0 0 0 0 0 0 1 1 1 1 0 0 0 0 1 0 1 1 1 0 1 0 0 0 0 0 1
 0 0 0 1 1 0 0 1 0 1 0 0 0 1 1 0 1 0 0 0 0 1 0 1 1 1 0 0 0 0 0 1 1 0 0 0 0
 1 0 1 0 0 0 0 0 1 1 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 0
 1 0 1 0 0 0 0 0 0 0 1 0 1 0 1 0 1 1 0 0 0 1 0 1 0 0 1 0 1 1 0 1 0 0 1 1 0
 0 1 0 0 1 1 0 0 0 0 0 0 1 1 0 1 0 0 0 0 1 1 0 0 0 1 0 1 0 0 1 0 1 1 0 0 0
 0 1 1 1 1 1 0 1 0 0 0]
----------------------------------------------------------------------------------------------------
{'C': 1}
0.7220903453925341


The below prints the probability for each row/person in the test set on their survival, specifically the probability that they did not survive

In [415]:
# Support Vector Machine
from sklearn.svm import SVC

svc = SVC()
svc.fit(X_train, y_train)
y_pred = svc.predict(X_test)
acc_svc = round(svc.score(X_train, y_train) * 100, 2)
acc_svc

82.83

In [416]:
# K nearest neighbour classifier
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
acc_knn = round(knn.score(X_train, y_train) * 100, 2)
acc_knn

85.07

In [417]:
# Decision Tree
from sklearn import tree

decision_tree = tree.DecisionTreeClassifier()
decision_tree.fit(X_train, y_train)
y_pred = decision_tree.predict(X_test)
acc_decision_tree = round(decision_tree.score(X_train, y_train) * 100, 2)
acc_decision_tree

87.21

In [418]:
from sklearn.ensemble import RandomForestClassifier

random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, y_train)
y_pred = random_forest.predict(X_test)
random_forest.score(X_train, y_train)
acc_random_forest = round(random_forest.score(X_train, y_train) * 100, 2)
acc_random_forest

87.21

In [413]:
y_pred

array([0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0,
       1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0,
       0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [422]:
submission = pd.DataFrame({
        "PassengerId": test_df["PassengerId"],
        "Survived": y_pred
    })
submission.to_csv('submission.csv', index=False)